In [ ]:
# PASO 1: Montar Google Drive
# Esto te permitirá acceder a los archivos almacenados en tu Drive.
# Se abrirá una ventana de autenticación para que permitas el acceso.
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=True) # force_remount puede ser útil si ya estaba montado
except Exception as e:
    print(f"Error al montar Google Drive: {e}")
    # Considerar detener el script si Drive no se monta
    # import sys
    # sys.exit("No se pudo montar Google Drive. El script no puede continuar.")

Mounted at /content/drive


# Extraer 4 filas Excel al Programa

In [ ]:
import pandas as pd
from openpyxl import load_workbook

def copiar_filas_celdas_especificas(archivo_excel, hoja_origen, hoja_destino, mapeo_celdas, filtro=None, num_filas=None, espaciado=8):
    """
    Copia múltiples filas de una hoja a celdas específicas en otra hoja.

    Parámetros:
    archivo_excel (str): Ruta del archivo Excel
    hoja_origen (str): Nombre de la hoja de origen
    hoja_destino (str): Nombre de la hoja de destino
    mapeo_celdas (dict): Diccionario que mapea columnas de origen a celdas de destino
    filtro (tuple): (columna, valor) para filtrar filas específicas
    num_filas (int): Número de últimas filas a copiar
    espaciado (int): Número de filas de espacio entre cada bloque de datos
    """
    try:
        # Leer la hoja de origen
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)

        # Aplicar filtro si se especifica
        if filtro:
            columna, valor = filtro
            df_filtrado = df_origen[df_origen[columna] == valor]
        else:
            df_filtrado = df_origen

        # Seleccionar las últimas n filas si se especifica
        if num_filas:
            filas_a_copiar = df_filtrado.tail(num_filas)
        else:
            filas_a_copiar = df_filtrado

        # Cargar el archivo con openpyxl
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]

        # Procesar cada fila
        for idx, fila in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)  # 18 es el rango original (19-2)

            # Copiar cada valor a su celda correspondiente
            for columna, celda_base in mapeo_celdas.items():
                # Calcular nueva posición
                letra = celda_base[0]
                numero = int(celda_base[1:]) + desplazamiento
                nueva_celda = f"{letra}{numero}"

                valor = fila[1][columna]
                hoja_dest[nueva_celda] = valor

        # Guardar los cambios
        wb.save(archivo_excel)
        print(f"Valores copiados exitosamente para {len(filas_a_copiar)} filas")

    except Exception as e:
        print(f"Error al copiar los valores: {str(e)}")





# Define el mapeo entre columnas de origen y celdas de destino
mapeo_celdas = {
    'SEMANA': 'B2',      # El valor de la columna 'Nombre' irá a la celda B2
    'PRESIDENCIA': 'O2',        # El valor de la columna 'Edad' irá a la celda D7
    'ORACIÓN': 'O3',
    'TESOROS DE LA BIBLIA': 'O5',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'O6',
    'LECTURA DE LA BIBLIA': 'O7',
    'SMM ASIG 1 ESTUD' : 'L9',
    'SMM ASIG 1 ACOMP' : 'O9',
    'SMM ASIG 2 ESTUD' : 'L10',
    'SMM ASIG 2 ACOMP' : 'O10',
    'SMM ASIG 3 ESTUD' : 'L11',
    'SMM ASIG 3 ACOMP' : 'O11',
    'SMM ASIG 4 ESTUD' : 'L12',
    'SMM ASIG 4 ACOMP' : 'O12',
    'NVC PARTE 1' : 'O15',
    'NVC PARTE 2' : 'O16',
    'ESUDIO LIBRO' : 'O17',
    'LECTOR LIBRO': 'O18',
    'ORACIÓN FINAL': 'O19'
}

# Ejemplo de uso para filtrar por mes

copiar_filas_celdas_especificas(
    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",
    hoja_origen="BD ASIG",
    hoja_destino="Formato",
    mapeo_celdas=mapeo_celdas,
    filtro=("MES", "MAYO")
)


# Ejemplo de uso para últimas 4 filas
"""
copiar_filas_celdas_especificas(
    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",
    hoja_origen="BD ASIG",
    hoja_destino="Formato",
    mapeo_celdas=mapeo_celdas,
    num_filas=4
)
"""

Valores copiados exitosamente para 4 filas


'\ncopiar_filas_celdas_especificas(\n    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",\n    hoja_origen="BD ASIG",\n    hoja_destino="Formato",\n    mapeo_celdas=mapeo_celdas,\n    num_filas=4\n)\n'

# Extracción de Datos de Web wol.jw.org - Semanas Reunión

In [ ]:

# ¡Importante! Modifica esta línea con tu ruta específica en Google Drive
base_path = '/content/drive/MyDrive/JW/CONGREGACIÓN/Ancianos/Super VMC/Programación-VMC/2025/Programador_VMC/Programación VMC_Septiembre-2024-2025.xlsx' # <--- ¡MODIFICA ESTA LÍNEA OBLIGATORIAMENTE!

# PASO 2: Instalar bibliotecas (si es necesario)
!pip install pandas openpyxl requests beautifulsoup4

# PASO 3: Importar bibliotecas
# ¡Asegúrate de ejecutar esta celda para importar todas las bibliotecas necesarias!
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import time
import re # <--- Módulo para expresiones regulares, ESENCIAL para la función extraer_informacion
import os # Para os.path.exists

print("Bibliotecas importadas, incluyendo 're'.")

# PASO 4: Definir nombres de archivo y URLs (adaptado de tu código)

# Nombre del archivo Excel principal y la hoja (se combinará con base_path)
nombre_archivo_principal = "Programación VMC_Septiembre-2024-2025.xlsx"
archivo_excel_definido_por_usuario = base_path + nombre_archivo_principal
nombre_hoja_definida_por_usuario = "Sheet1"

# Nombres para archivos de respaldo y nuevos (se combinarán con base_path)
archivo_backup_excel = base_path + "Info_Reuniones_Backup.xlsx"
archivo_nuevas_reuniones_excel = base_path + "Info_Reuniones_Nuevas.xlsx"

# Lista de URLs de tu código original
# Asegúrate que estas sean las URLs correctas que quieres procesar.
# El año 2025 se usa como en tu ejemplo.
urls_a_scrapear = [
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/23",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/24",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/25",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/26",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/27" # Asumo que la última repetida era un error y la cambié a 27
]
print(f"Archivo Excel principal esperado en: {archivo_excel_definido_por_usuario}")
print("\nURLs a scrapear:")
for url in urls_a_scrapear:
    print(url)

# PASO 5: Tu función original para extraer información
def extraer_informacion(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Añadimos un try-except para la solicitud HTTP por si falla una URL
    try:
        response = requests.get(url, headers=headers, timeout=20) # Timeout de 20s
        response.raise_for_status() # Lanza error para respuestas 4xx/5xx
    except requests.exceptions.RequestException as e_req:
        print(f"  Error de conexión/HTTP para {url}: {e_req}")
        return None # Retornar None si la URL falla para que el bucle principal lo maneje

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extracciones existentes...
    semana_element = soup.find('h1', id='p1')
    semana = semana_element.get_text(strip=True) if semana_element else "Semana no encontrada"

    libro_element = soup.find('h2', id='p2')
    libro = "Libro no encontrado"
    if libro_element:
        strong_tag = libro_element.find('strong')
        if strong_tag:
            libro = strong_tag.get_text(strip=True)
        else:
            libro = libro_element.get_text(strip=True)

    cancion_element = soup.find('h3', id='p3')
    cancion_text = "" # Inicializar para evitar error si cancion_element es None
    cancion = "Canción no encontrada"
    if cancion_element:
        cancion_text = cancion_element.get_text(strip=True)
        # Uso del módulo 're' para buscar patrones en el texto de la canción
        match = re.search(r'Canción\s+(\d+)', cancion_text) # Aquí se usa 're'
        if match:
            cancion = f"Canción {match.group(1)}"
        # Si no hay número pero dice "Canción", tomar el texto tal cual
        elif "Canción" in cancion_text:
            cancion = cancion_text

    tesoros_element = soup.find(['h5', 'h4', 'h3'], id='p5')
    tesoros = tesoros_element.get_text(strip=True) if tesoros_element else "Tesoros no encontrados"

    # Extraer todas las canciones como antes...
    todas_h3 = soup.find_all('h3')
    segunda_cancion = "Segunda canción no encontrada"
    # Usar cancion_text (el texto original de la primera canción) para la comparación
    for h3_tag in todas_h3:
        texto_h3 = h3_tag.get_text(strip=True)
        if "Canción" in texto_h3 and texto_h3 != cancion_text: # Comparar con el texto original de la primera canción
            match = re.search(r'Canción\s+(\d+)', texto_h3) # Aquí se usa 're'
            if match:
                segunda_cancion = f"Canción {match.group(1)}"
            elif "Canción" in texto_h3: # Si no hay número pero dice "Canción"
                 segunda_cancion = texto_h3
            break # Encontrar la primera diferente y salir

    tercera_cancion = "Tercera canción no encontrada"
    # Para la tercera canción, debe ser diferente de la primera Y de la segunda.
    for h3_tag in reversed(todas_h3): # Buscar desde el final
        texto_h3 = h3_tag.get_text(strip=True)
        # Asegurarse que el texto_h3 no esté vacío y sea diferente de las otras dos canciones encontradas
        if "Canción" in texto_h3 and texto_h3 and texto_h3 != cancion_text and texto_h3 != segunda_cancion:
            match = re.search(r'Canción\s+(\d+)', texto_h3) # Aquí se usa 're'
            if match:
                tercera_cancion = f"Canción {match.group(1)}"
            elif "Canción" in texto_h3: # Si no hay número pero dice "Canción"
                 tercera_cancion = texto_h3
            break # Encontrar la primera diferente (desde el final) y salir

    # Extraer títulos h3 entre SEAMOS MEJORES MAESTROS y NUESTRA VIDA CRISTIANA
    titulos_maestros = []
    maestros_element = None
    vida_cristiana_element = None

    for h2_tag in soup.find_all('h2'):
        texto_h2 = h2_tag.get_text(strip=True)
        if "SEAMOS MEJORES MAESTROS" in texto_h2:
            maestros_element = h2_tag
        elif "NUESTRA VIDA CRISTIANA" in texto_h2:
            vida_cristiana_element = h2_tag

    if maestros_element and vida_cristiana_element:
        elemento_actual = maestros_element.next_sibling
        while elemento_actual and elemento_actual != vida_cristiana_element:
            if hasattr(elemento_actual, 'name') and elemento_actual.name == 'h3':
                strong_tag_maestros = elemento_actual.find('strong')
                if strong_tag_maestros:
                    strong_text = strong_tag_maestros.get_text(strip=True)
                    if not strong_text.startswith("Canción"):
                        titulos_maestros.append(strong_text)
            try:
                elemento_actual = elemento_actual.next_sibling
            except AttributeError: # Algunos elementos como NavigableString no tienen .name o .next_sibling
                # Intento de manejo más robusto para elementos sin 'next_sibling' o 'name'
                if hasattr(elemento_actual, 'next_element'):
                     elemento_actual = elemento_actual.next_element
                     if elemento_actual == vida_cristiana_element: # Chequeo adicional
                         break
                else: # Si no hay forma de avanzar, salir del bucle
                     break


    # NUEVO: Extraer títulos h3 dentro de NUESTRA VIDA CRISTIANA hasta "Estudio bíblico de la congregación"
    titulos_nvc = []
    estudio_biblico_encontrado = False
    estudio_biblico_texto = None # Inicializar como None

    if vida_cristiana_element:
        elemento_actual = vida_cristiana_element.next_sibling
        while elemento_actual:
            if hasattr(elemento_actual, 'name') and elemento_actual.name == 'h3':
                strong_tag_nvc = elemento_actual.find('strong')
                if strong_tag_nvc:
                    strong_text = strong_tag_nvc.get_text(strip=True)
                    if "Estudio bíblico de la congregación" in strong_text:
                        estudio_biblico_encontrado = True
                        estudio_biblico_texto = strong_text # Guardar el texto exacto
                        break
                    elif not strong_text.startswith("Canción"):
                        titulos_nvc.append(strong_text)
            try:
                elemento_actual = elemento_actual.next_sibling
            except AttributeError:
                if hasattr(elemento_actual, 'next_element'):
                     elemento_actual = elemento_actual.next_element
                else:
                     break

    if not estudio_biblico_encontrado: # Búsqueda global si no se encontró en la sección
        for h3_tag_estudio in soup.find_all('h3'): # Renombrar variable para evitar confusión
            strong_tag_estudio_glob = h3_tag_estudio.find('strong') # Renombrar variable
            if strong_tag_estudio_glob and "Estudio bíblico de la congregación" in strong_tag_estudio_glob.get_text(strip=True):
                estudio_biblico_encontrado = True
                estudio_biblico_texto = strong_tag_estudio_glob.get_text(strip=True) # Guardar el texto exacto
                break

    nvc_titulo1 = titulos_nvc[0] if titulos_nvc else ""
    nvc_titulo2 = titulos_nvc[1] if len(titulos_nvc) > 1 else ""
    # Usar el texto encontrado o un default si sigue sin encontrarse
    nvc_titulo3 = estudio_biblico_texto if estudio_biblico_encontrado and estudio_biblico_texto else "Estudio bíblico de la congregación (No encontrado)"

    return {
        "semana": semana, "libro": libro, "cancion_inicial": cancion,
        "tesoros": tesoros, "segunda_cancion": segunda_cancion,
        "tercera_cancion": tercera_cancion, "titulos_maestros": titulos_maestros,
        "nvc_titulo1": nvc_titulo1, "nvc_titulo2": nvc_titulo2, "nvc_titulo3": nvc_titulo3
    }

# PASO 6: Procesar cada URL y recolectar datos
lista_de_datos_extraidos = []

for url_actual in urls_a_scrapear:
    print(f"\nProcesando: {url_actual}")
    info = extraer_informacion(url_actual) # Llama a tu función

    if info: # Solo procesar si la extracción fue exitosa
        datos_para_df = {
            "Semana": info["semana"],
            "Libro": info["libro"],
            "Canción Inicial": info["cancion_inicial"],
            "Tesoros de la Biblia": info["tesoros"],
            "Segunda Canción": info["segunda_cancion"],
            "Canción Final": info["tercera_cancion"]
        }

        # Asegurarse de que hay suficientes títulos antes de acceder por índice
        for i in range(max(0, len(info.get("titulos_maestros", [])))): # Usar .get con default
            datos_para_df[f"Maestros Título {i+1}"] = info["titulos_maestros"][i]

        datos_para_df["NVC Título 1"] = info.get("nvc_titulo1", "") # Usar .get con default
        datos_para_df["NVC Título 2"] = info.get("nvc_titulo2", "") # Usar .get con default
        datos_para_df["NVC Título 3"] = info.get("nvc_titulo3", "") # Usar .get con default

        lista_de_datos_extraidos.append(datos_para_df)

        # Mostrar información detallada (opcional, como en tu código)
        print("  --- Información Extraída ---")
        for clave, valor in info.items():
            if clave != "titulos_maestros":
                print(f"  {clave.replace('_', ' ').capitalize()}: {valor}")
            else:
                print(f"  Títulos de 'Seamos Mejores Maestros' ({len(valor)}):")
                for idx, tit_m in enumerate(valor, 1):
                    print(f"    {idx}. {tit_m}")
        print("  --------------------------")
    else:
        print(f"  No se pudo extraer información de {url_actual}. Saltando.")

    time.sleep(1) # Pausa entre solicitudes

# PASO 7: Crear DataFrame
if lista_de_datos_extraidos:
    df_final = pd.DataFrame(lista_de_datos_extraidos)
    print("\n--- Resumen de información extraída (DataFrame) ---")
    # Imprimir más filas si es un DF pequeño, o head/tail para DFs grandes
    if len(df_final) < 10:
        print(df_final)
    else:
        print(df_final.head())
else:
    print("\nNo se extrajo ningún dato. El DataFrame estará vacío.")
    df_final = pd.DataFrame() # Crear DataFrame vacío para evitar errores al guardar

# PASO 8: Guardar en Excel (lógica adaptada de tu código)
if not df_final.empty:
    if os.path.exists(archivo_excel_definido_por_usuario):
        try:
            print(f"\nIntentando agregar/reemplazar la hoja '{nombre_hoja_definida_por_usuario}' en '{archivo_excel_definido_por_usuario}'...")
            with pd.ExcelWriter(archivo_excel_definido_por_usuario,
                                engine='openpyxl',
                                mode='a',
                                if_sheet_exists='replace') as writer:
                df_final.to_excel(writer, sheet_name=nombre_hoja_definida_por_usuario, index=False)
            print(f"Datos agregados/reemplazados exitosamente en la hoja '{nombre_hoja_definida_por_usuario}'.")

        except Exception as e:
            print(f"Error al agregar datos al archivo Excel existente '{archivo_excel_definido_por_usuario}': {e}")
            print(f"Guardando datos en archivo de respaldo: '{archivo_backup_excel}'")
            try:
                df_final.to_excel(archivo_backup_excel, index=False)
                print(f"Datos guardados en archivo de respaldo: '{archivo_backup_excel}'")
            except Exception as e_backup:
                print(f"Error al guardar el archivo de respaldo: {e_backup}")
    else:
        print(f"¡Atención! El archivo '{archivo_excel_definido_por_usuario}' no existe.")
        print(f"Guardando datos en un nuevo archivo: '{archivo_nuevas_reuniones_excel}'")
        try:
            df_final.to_excel(archivo_nuevas_reuniones_excel, index=False)
            print(f"Datos guardados en un nuevo archivo: '{archivo_nuevas_reuniones_excel}'")
        except Exception as e_new:
            print(f"Error al guardar el nuevo archivo: {e_new}")
else:
    print("\nNo hay datos para guardar en el archivo Excel.")

# Mostrar la tabla final (opcional)
if not df_final.empty:
    print("\n--- Contenido Final del DataFrame ---")
    if len(df_final) < 10:
        print(df_final)
    else:
        print(df_final.to_string()) # .to_string() para mostrar más contenido si es largo


# PASO 9: Desmontar Google Drive (opcional)
# try:
#     drive.flush_and_unmount()
#     print('\nGoogle Drive desmontado.')
# except Exception as e:
#     print(f"Error al intentar desmontar Google Drive: {e}")


Bibliotecas importadas, incluyendo 're'.
Archivo Excel principal esperado en: /content/drive/MyDrive/JW/CONGREGACIÓN/Ancianos/Super VMC/Programación-VMC/2025/Programador_VMC/Programación VMC_Septiembre-2024-2025.xlsxProgramación VMC_Septiembre-2024-2025.xlsx

URLs a scrapear:
https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/23
https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/24
https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/25
https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/26
https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/27

Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/23
  --- Información Extraída ---
  Semana: 2-8 DE JUNIO
  Libro: PROVERBIOS 16
  Cancion inicial: Canción 36
  Tesoros: 1. Tres preguntas que lo ayudarán a tomar buenas decisiones
  Segunda cancion: Canción 32
  Tercera cancion: Canción 68
  Títulos de 'Seamos Mejores Maestros' (0):
  Nvc titulo1: 
  Nvc titulo2: 
  Nvc titulo3: 8. Estudio bíblico de la congregación
  --------------------------

Proc

# Copiar informacón de la reunión

In [8]:


import pandas as pd
from openpyxl import load_workbook
from openpyxl.cell.cell import MergedCell

def copiar_info_reunion(archivo_excel, hoja_origen="Info-reunión", hoja_destino="Formato", espaciado=8):
    """
    Copia la información de reuniones de la hoja Info-reunión a celdas específicas en la hoja Formato,
    manejando correctamente las celdas combinadas.

    Parámetros:
    archivo_excel (str): Ruta del archivo Excel
    hoja_origen (str): Nombre de la hoja que contiene la información de reuniones
    hoja_destino (str): Nombre de la hoja donde se copiará la información
    espaciado (int): Número de filas de espacio entre cada bloque de datos
    """
    try:
        # Leer la hoja de origen
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)

        # Cargar el archivo con openpyxl
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]

        # Definir el mapeo de columnas a celdas
        mapeo_celdas = {
            'Semana': 'B2',           # Semana
            'Libro': 'I2',            # Libro de estudio
            'Canción Inicial': 'D3',  # Primera canción
            'Tesoros de la Biblia': 'C5',  # Tesoros
            'Segunda Canción': 'D14',      # Segunda canción
            'Canción Final': 'D19',         # Tercera canción
            'Maestros Título 1': 'C9',     # Asignación de maestros 1
            'Maestros Título 2': 'C10',    # Asignación de maestros 2
            'Maestros Título 3': 'C11',    # Asignación de maestros 3
            "NVC Título 1": "C15",    # Asignación de vida cristiana 1
            "NVC Título 2": "C16",    # Asignación de vida cristiana 2
            "NVC Título 3": "C17",    # Asignación de vida cristiana 3
        }

        def encontrar_celda_principal(hoja, celda_combinada):
            """
            Encuentra la celda principal de un rango de celdas combinadas.
            """
            for rango in hoja.merged_cells.ranges:
                if celda_combinada.coordinate in rango:
                    return hoja.cell(row=rango.min_row, column=rango.min_col)
            return celda_combinada

        # Procesar cada fila
        for idx, fila in enumerate(df_origen.iterrows()):
            desplazamiento = idx * (espaciado + 18)

            # Copiar cada valor a su celda correspondiente
            for columna, celda_base in mapeo_celdas.items():
                # Calcular nueva posición
                letra = celda_base[0]
                numero = int(celda_base[1:]) + desplazamiento
                nueva_celda = f"{letra}{numero}"

                celda = hoja_dest[nueva_celda]
                valor = fila[1][columna]

                # Si es una celda combinada, encontrar la celda principal
                if isinstance(celda, MergedCell):
                    celda_principal = encontrar_celda_principal(hoja_dest, celda)
                    if celda_principal:
                        celda_principal.value = valor
                else:
                    celda.value = valor

        # Guardar los cambios
        wb.save(archivo_excel)
        print(f"Información copiada exitosamente para {len(df_origen)} semanas")

    except Exception as e:
        print(f"Error al copiar la información: {str(e)}")
    finally:
        wb.close()

# Ejemplo de uso
if __name__ == "__main__":
    archivo = "Programación VMC_Septiembre-2024-2025.xlsx"
    copiar_info_reunion(archivo)

Error al copiar la información: 'Maestros Título 1'
